In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from PyPDF2 import PdfReader
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings


In [5]:
client = OpenAI(api_key=os.getenv('API_KEY'))
openai = OpenAIEmbeddings(openai_api_key=os.getenv('API_KEY'))

In [ ]:
pdf = PdfReader("tabagismo.pdf")

In [7]:
raw_text =''
for i, page in enumerate(pdf.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
print(raw_text)

In [10]:
# A intuito dessa função é fatiar o texto contido na variavel raw_text.

texto = raw_text

# Pontos de Fatiamento
termos = [
    '\n1. ', '\n2. ', '\n3. ', '\n4. ', '\n5. ',
    '\n6. ', '\n7. ', '\n8. ', '\n9. ', '\n10. ',
    '\n11. '
]

def fatiar_texto(texto, termos):
    partes = []
    inicio = 0
    for termo in termos:
        fim = texto.find(termo, inicio + 1)
        partes.append(texto[inicio:fim])
        inicio = fim
    partes.append(texto[inicio:])
    return partes

partes_texto = fatiar_texto(texto, termos)


In [ ]:
# Mostrar fatiamentos
"""# Impressão das partes fatiadas
for i, parte in enumerate(partes_texto):
    print(f"Parte {i + 1}:")
    print(parte.strip())  # strip() para remover espaços em branco extras
    print("-" * 50)  # linha separadora"""

In [23]:
partes_texto

[' \n \nMINISTÉRIO DA SAÚDE  \nSECRETARIA DE ATENÇÃO ESPECIALIZADA À SAÚDE  \nSECRETARIA DE CIÊNCIA, TECNOLOGIA, INOVAÇÃO E INSUMOS ESTRATÉGICOS \nEM SAÚDE  \n \n \n \n \n \nPORTARIA CONJUNTA Nº 10, DE 16 DE ABRIL DE 2020  \n \nAprova o Protocolo Clínico e \nDiretrizes Terapêuticas do Tabagismo.  \n \n \nO SECRETÁRIO DE ATENÇÃO ESPECIALIZADA À SAÚDE  e o \nSECRETÁRIO DE CIÊNCIA, TECNOLOGIA, INOVAÇÃO E INSUMOS \nESTRATÉGICOS EM SAÚDE , no uso das atribuições,  \n \nConsiderando a necessidade  de se atualizarem parâmetros sobre o tabagismo \nno Brasil e diretrizes nacionais para diagnóstico, tratamento e acompanhamento dos \nindivíduos com esta doença;  \nConsiderando que os protocolos clínicos e diretrizes terapêuticas são \nresultado de consenso técni co-científico e são formulados dentro de rigorosos parâmetros de \nqualidade e precisão de indicação;  \nConsiderando a Portaria Nº 571 /GM/MS , de 5 de abril de 2013, que atualiza \nas diretrizes de cuidado à pessoa tabagista no âmbito d

In [14]:
doc0 = Document(page_content=partes_texto[0], metadata={'source': 'tabagismo.pdf', 'page':0 })
doc1 = Document(page_content=partes_texto[1], metadata={'source': 'tabagismo.pdf', 'page':1 })
doc2 = Document(page_content=partes_texto[2], metadata={'source': 'tabagismo.pdf', 'page':2 })
doc3 = Document(page_content=partes_texto[3], metadata={'source': 'tabagismo.pdf', 'page':3 })
doc4 = Document(page_content=partes_texto[4], metadata={'source': 'tabagismo.pdf', 'page':4 })
doc5 = Document(page_content=partes_texto[5], metadata={'source': 'tabagismo.pdf', 'page':5 })
doc6 = Document(page_content=partes_texto[6], metadata={'source': 'tabagismo.pdf', 'page':6 })
doc7 = Document(page_content=partes_texto[7], metadata={'source': 'tabagismo.pdf', 'page':7 })
doc8 = Document(page_content=partes_texto[8], metadata={'source': 'tabagismo.pdf', 'page':8 })
doc9 = Document(page_content=partes_texto[9], metadata={'source': 'tabagismo.pdf', 'page':9 })
doc10 = Document(page_content=partes_texto[10], metadata={'source': 'tabagismo.pdf', 'page':10 })
doc11 = Document(page_content=partes_texto[11], metadata={'source': 'tabagismo.pdf', 'page':11 })


In [15]:
list_doc=[doc0,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10,doc11]

In [16]:
db = FAISS.from_documents(list_doc, openai)
db.save_local('store_path')

In [17]:
print(db.index.ntotal)

12


In [20]:
query = "Em media quantos cigarros fuma por dia"
docs = db.similarity_search(query)

In [21]:
print(docs)

[Document(page_content='\n4. Quantos cigarros você fuma por dia?  Menos de 10  0 \n 11-20 1 \n 21-30 2 \n Mais de  31 3 ', metadata={'source': 'tabagismo.pdf', 'page': 4}), Document(page_content='\n5. Você fuma mais frequentemente pela manhã?  Sim 1 \n Não 0 ', metadata={'source': 'tabagismo.pdf', 'page': 5}), Document(page_content='\n6. Você fuma mesmo doente, quando precisa ficar  Sim 1 \nacamado a maior parte do tempo?  Não 0 \nEscore Total  (da dependência) : 0-2 = muito baixa; 3 -4 = baixa; 5 = média; 6 -7 = elevada; 8 -10 = muito elevada  \n \nO teste de Fagerström não engloba todas as situações possíveis da rotina do \nfumante, como , por exemplo: indivíduos que adotaram outras formas de uso em \ndecorrência da legislação sobre Ambientes 100% Livres da Fumaça de Tabaco e fumam \nem outros locais e horários, para evitar o desconforto da síndrome de abstinência \nprolongada; indivíduos que fum am mais no período da noite ou interrompem o sono para \nfumar para sair da síndrome de 